Iterative Solvers
===

So far we have used direct solvers to solve the linear system of equations. Although a direct solver can profit from the sparse matrix, it's arithmetic complexity is sub-optimal. For large-scale problems iterative solvers are a must.

The conjugate gradient (cg) method is the standard method for symmetric and positive definite matrices. It's convergence rate depends on a preconditioner, what is a cheap approximative inverse to the matrix.

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

We generate a 3D geometry and mesh using the OCC constructive solid geometry (CSG) modeler:

In [2]:
from netgen.occ import *
cube = Box((0,0,0),(1,1,1))
cyl = Cylinder((0,0.5,0.5),X, r=0.2, h=1)
cube.faces.name = "outer"
cyl.faces.name = "cyl"
shape = cube-cyl
DrawGeo(shape);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

In [3]:
ngmesh = OCCGeometry(shape).GenerateMesh(maxh=0.1)
for l in range(0):
    ngmesh.Refine()
mesh = Mesh(ngmesh)
mesh.Curve(3)
Draw (mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

In [4]:
fes = H1(mesh, order=3, dirichlet="outer", wb_withedges=False)
print ("we have", fes.ndof, "unknowns")
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes)
a += grad(u)*grad(v)*dx

f = LinearForm(fes)
f += v*dx

# c = Preconditioner(a, "direct", inverse="sparsecholesky")
c = Preconditioner(a, "local")
# c = Preconditioner(a, "bddc")

gfu = GridFunction(fes)

we have 24122 unknowns


assemble system and setup preconditioner in parallel:

In [5]:
ngsglobals.msg_level=5
with TaskManager():
    a.Assemble()
    f.Assemble()

assemble VOL element 4508/4508                                 
assemble VOL element 4508/4508                                 


solve the system using the preconditioned conjugate gradient method:

In [6]:
from ngsolve.krylovspace import CGSolver

with TaskManager():
    inv = CGSolver(mat=a.mat, pre=c.mat, printrates=True, maxiter=400)
    gfu.vec.data = inv * f.vec

CG iteration 1, residual = 0.04806510211955605     
CG iteration 2, residual = 0.053444916495502315     
CG iteration 3, residual = 0.05597869305039399     
CG iteration 4, residual = 0.04586947132392968     
CG iteration 5, residual = 0.030649990803653278     
CG iteration 6, residual = 0.024350139933223672     
CG iteration 7, residual = 0.01897169465754093     
CG iteration 8, residual = 0.013456673742780959     
CG iteration 9, residual = 0.009691378582119851     
CG iteration 10, residual = 0.007602921521578274     
CG iteration 11, residual = 0.006742906370794676     
CG iteration 12, residual = 0.0064356871493630365     
CG iteration 13, residual = 0.0063719120543688675     
CG iteration 14, residual = 0.006727514653593267     
CG iteration 15, residual = 0.006627266907791014     
CG iteration 16, residual = 0.005696907354487289     
CG iteration 17, residual = 0.004423754350764521     
CG iteration 18, residual = 0.0033608706866222267     
CG iteration 19, residual = 0.00253659

In [7]:
Draw (gfu);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…